# Assigment 6


This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

**GET YOUR GOOGLE API AND TOKEN. YOU WILL NEED THEM TO DO THIS TASK.**


1. Import Data from [this url](https://github.com/alexanderquispe/Diplomado_PUCP/blob/main/_data/bbva_list.xlsx). This dataset is in excel format. You have to convert to PandasDataFrame.
2. Use GoogleMaps API and geocode all the BBVA offices. For those offices that Google API gets no information, use internet and get the latitude and longitude handly and add them to dataset.
3. Use Google API to find the driving time (best guess) from all the group members' address and all the LIMA BBVA offices.
4. Finally, you have to give a report which offices are the most closest and furthest to every group member's address.

In [ ]:
# Charging the libraries
!pip install -U googlemaps

In [1]:
# Packages
import pandas as pd
import numpy as np
import urllib.request, json, csv
import googlemaps

from tqdm import tqdm_notebook as tqdm
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time
import requests

# Cargamos el archivo

In [2]:
bbva_offices = pd.read_excel('../../_data/bbva_list.xlsx')
bbva_offices

,Direccion,DEPARTAMENTO,PROVINCIA,DISTRITO
0,CENTRO AEREO COMERCIAL LOCALES 110 A Y 111 A,LIMA,LIMA,CALLAO
1,AV. CTRMTE. MORA S/N BASE NAVAL,LIMA,LIMA,CALLAO
2,"AV. ELMER FAUCETT Y ALEJANDRO BERTELLO, CC CAN...",LIMA,LIMA,CALLAO
3,AV. SAENZ PEN A 323,LIMA,LIMA,CALLAO
4,CALLE OMEGA 149 PARQUE INDUSTRIAL DEL CALLAO,LIMA,LIMA,CALLAO
5,AV. CONTRALMIRANTE RAYGADA N°lll,LIMA,LIMA,CALLAO
6,"AV. ELMERT FAUCETT N°2121 LOCALES N° 2-101,2-1...",LIMA,LIMA,CALLAO
7,AV. ELMER FAUCETT 6000,LIMA,LIMA,CALLAO
8,CENTRO COMERCIAL MINKA PABELL6N 2,LIMA,LIMA,CALLAO
9,"AV. OSCAR R. BENAVIDES 3866, URB. EL AGUILA, L...",LIMA,LIMA,CALLAO


In [3]:
bbva_offices.rename(columns={'Direccion': 'DIRECCIÓN'}, inplace=True)
# iloc usa el índice de la columna como identificador (no la etiqueta) y al indicar un rango no toma el último valor (1:4 no toma el 4)
# loc usa la etiqueta de la columna como identificador (no el índice) y al indicar el rango sí toma el último valor
bbva_offices.iloc[23:26, 3], bbva_offices.iloc[36, 3], bbva_offices.iloc[40:45, 3] = 'BREÑA', 'EL AGUSTINO', 'JESÚS MARÍA'
bbva_offices

,DIRECCIÓN,DEPARTAMENTO,PROVINCIA,DISTRITO
0,CENTRO AEREO COMERCIAL LOCALES 110 A Y 111 A,LIMA,LIMA,CALLAO
1,AV. CTRMTE. MORA S/N BASE NAVAL,LIMA,LIMA,CALLAO
2,"AV. ELMER FAUCETT Y ALEJANDRO BERTELLO, CC CAN...",LIMA,LIMA,CALLAO
3,AV. SAENZ PEN A 323,LIMA,LIMA,CALLAO
4,CALLE OMEGA 149 PARQUE INDUSTRIAL DEL CALLAO,LIMA,LIMA,CALLAO
5,AV. CONTRALMIRANTE RAYGADA N°lll,LIMA,LIMA,CALLAO
6,"AV. ELMERT FAUCETT N°2121 LOCALES N° 2-101,2-1...",LIMA,LIMA,CALLAO
7,AV. ELMER FAUCETT 6000,LIMA,LIMA,CALLAO
8,CENTRO COMERCIAL MINKA PABELL6N 2,LIMA,LIMA,CALLAO
9,"AV. OSCAR R. BENAVIDES 3866, URB. EL AGUILA, L...",LIMA,LIMA,CALLAO


# Definimos la función para obtener las coordenadas a partir de la dirección completa

In [ ]:
gmaps = googlemaps.Client(key = 'PONER NEW KEY API')
## Ejemplo de cómo se ve el diccionario que obtenemos al usar geocode con una dirección
example_1 = gmaps.geocode( "Av. Universitaria 18 - Interior PUCP, San Miguel" , region='PE')
print(example_1); print()
# El output es una lista que contiene diccionarios y listas anidados 
print(len(example_1)); print()
print(f'Aquí visualizamos el primer elemento de la lista\n'example_1[0]); print()
print(f'Aquí generamos una lista con las llaves del diccionario\n', list(example_1[0].keys())); print()
print(f'Aquí generamos una lista con las llaves del diccionario "geometry"\n', list(example_1[0]['geometry'])); print()
print(f'Aquí generamos una lista con las llaves del diccionario "location"\n', list(example_1[0]['geometry']['location'])); print()
# Entonces, el formato para obtener las coordenadas (lat, long) será:
# result_API[0]['geometry']['location']
# Donde result_API es el output de geocode (de preferencia, un archivo 'json')

In [4]:


def geo_bbva(row_series):
    
    Ubicación = ', '.join(map(row_series.get, ['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'DIRECCIÓN']))

    # Set Geolocation
    result_api = gmaps.geocode(Ubicación, region = 'PE')
    
    # Information
    try:
        lat = result_api[0]['geometry']['location']['lat']
        lon = result_api[0]['geometry']['location']['lng']   
    except:
        lat = np.nan
        lon = np.nan
    
    return lat, lon

In [5]:
bbva_offices['COORDENADAS: Lat, Long'] = bbva_offices.apply(geo_bbva, axis=1)
# Para reordenar las columnas
bbva_offices = bbva_offices[['DIRECCIÓN', 'DISTRITO', 'PROVINCIA', 'DEPARTAMENTO', 'COORDENADAS: Lat, Long']]
bbva_offices

,DIRECCIÓN,DISTRITO,PROVINCIA,DEPARTAMENTO,"COORDENADAS: Lat, Long"
0,CENTRO AEREO COMERCIAL LOCALES 110 A Y 111 A,CALLAO,LIMA,LIMA,"(-12.0244324, -77.1041764)"
1,AV. CTRMTE. MORA S/N BASE NAVAL,CALLAO,LIMA,LIMA,"(-12.0511717, -77.1256883)"
2,"AV. ELMER FAUCETT Y ALEJANDRO BERTELLO, CC CAN...",CALLAO,LIMA,LIMA,"(-12.0309896, -77.1014989)"
3,AV. SAENZ PEN A 323,CALLAO,LIMA,LIMA,"(-12.0511717, -77.1256883)"
4,CALLE OMEGA 149 PARQUE INDUSTRIAL DEL CALLAO,CALLAO,LIMA,LIMA,"(-12.0506798, -77.0871366)"
5,AV. CONTRALMIRANTE RAYGADA N°lll,CALLAO,LIMA,LIMA,"(-12.0464501, -77.1400374)"
6,"AV. ELMERT FAUCETT N°2121 LOCALES N° 2-101,2-1...",CALLAO,LIMA,LIMA,"(-12.0379707, -77.0986351)"
7,AV. ELMER FAUCETT 6000,CALLAO,LIMA,LIMA,"(-11.9972712, -77.1245194)"
8,CENTRO COMERCIAL MINKA PABELL6N 2,CALLAO,LIMA,LIMA,"(-12.048302, -77.10948069999999)"
9,"AV. OSCAR R. BENAVIDES 3866, URB. EL AGUILA, L...",CALLAO,LIMA,LIMA,"(-12.0548677, -77.10383379999999)"


# PARTE 3: CALCULAR LA DISTANCIA ENTRE LA DIRECCIÓN DE CADA INTEGRANTE DEL GRUPO Y CADA OFICINA DEL BBVA

In [10]:
G3_coord = {
    'Integrante': [ 'Mauricio', 'Reynaldo', 'Vania'],
    'Geocode_Dom': [ (-12.0614576, -77.0468779), (-11.976752910871854, -77.05952789115892), (-12.068858, -77.078141)]
}

G3_df = pd.DataFrame(G3_coord)
G3_df

,Integrante,Geocode_Dom
0,Mauricio,"(-12.0614576, -77.0468779)"
1,Reynaldo,"(-11.976752910871854, -77.05952789115892)"
2,Vania,"(-12.068858, -77.078141)"


In [7]:
# Definir una función para calcular el tiempo de conducción
def driving_time(origin, destination):
    result = gmaps.distance_matrix(
        origins=origin, 
        destinations=destination, 
        mode='driving', 
        region='PE',
        language= 'es',
        traffic_model='best_guess',
        departure_time='now'
    )
    try:
        dist_segundos = result['rows'][0]['elements'][0]['duration']['value']
        time = f'{round(dist_segundos / 60)} min'  # Convertir segundos a minutos
    except:
        time = np.nan
    return time